In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from torch.autograd import Variable
import torch

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/breast-cancer-wisconsin-data/data.csv


In [2]:
data=pd.read_csv('../input/breast-cancer-wisconsin-data/data.csv')
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
cols=data.columns
x_data=data[cols[2:-1]]
x_data.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
y_data=data[cols[1]]
l=LabelEncoder()
y_data=np.array(l.fit_transform(y_data))
print(y_data[:5])
l.classes_

[1 1 1 1 1]


array(['B', 'M'], dtype=object)

In [5]:
x_data = Variable(torch.from_numpy(x_data.values))
y_data = Variable(torch.from_numpy(y_data))

In [6]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.linearLayer1=torch.nn.Linear(30,16)
        self.linearLayer2=torch.nn.Linear(16,4)
        self.linearLayer3=torch.nn.Linear(4,1)
        
        self.sigmoid=torch.nn.Sigmoid()
        
    def forward(self,x):
        output1=self.sigmoid(self.linearLayer1(x))
        output2=self.sigmoid(self.linearLayer2(output1))
        y_pred=self.sigmoid(self.linearLayer3(output2))
        return(y_pred) 
        

In [7]:
model=Model()
print(model.parameters)
lossFunction=torch.nn.BCELoss(reduction="sum")
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

<bound method Module.parameters of Model(
  (linearLayer1): Linear(in_features=30, out_features=16, bias=True)
  (linearLayer2): Linear(in_features=16, out_features=4, bias=True)
  (linearLayer3): Linear(in_features=4, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)>


In [8]:
def train():
    for epochs in range(500):
        y_pred=model(x_data.float())
        loss=lossFunction(y_pred,y_data.view(-1,1).float())
        print('Epoch',epochs,'Loss:',loss.item(), '- Pred:', y_pred.data[0])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
train()
        

Epoch 0 Loss: 402.23309326171875 - Pred: tensor([0.5296])
Epoch 1 Loss: 401.71160888671875 - Pred: tensor([0.5286])
Epoch 2 Loss: 401.217041015625 - Pred: tensor([0.5275])
Epoch 3 Loss: 400.74615478515625 - Pred: tensor([0.5265])
Epoch 4 Loss: 400.29388427734375 - Pred: tensor([0.5255])
Epoch 5 Loss: 399.833984375 - Pred: tensor([0.5242])
Epoch 6 Loss: 399.3901672363281 - Pred: tensor([0.5217])
Epoch 7 Loss: 398.92669677734375 - Pred: tensor([0.5200])
Epoch 8 Loss: 398.4705810546875 - Pred: tensor([0.5190])
Epoch 9 Loss: 398.0189514160156 - Pred: tensor([0.5180])
Epoch 10 Loss: 397.5652770996094 - Pred: tensor([0.5170])
Epoch 11 Loss: 397.12188720703125 - Pred: tensor([0.5161])
Epoch 12 Loss: 396.6869812011719 - Pred: tensor([0.5151])
Epoch 13 Loss: 396.2550354003906 - Pred: tensor([0.5141])
Epoch 14 Loss: 395.8239440917969 - Pred: tensor([0.5132])
Epoch 15 Loss: 395.38861083984375 - Pred: tensor([0.5122])
Epoch 16 Loss: 394.9493103027344 - Pred: tensor([0.5112])
Epoch 17 Loss: 394.530

In [9]:
model.double().forward(x_data.data[21]) > 0.5 , y_data[25]  # This is generating the output class for one instance of data

(tensor([False]), tensor(1))

In [10]:
pred = model.double().forward(x_data) > 0.5
pred.numpy()[:5]

#These are all the predictions weather the cancers are malignant or benign

array([[ True],
       [ True],
       [ True],
       [False],
       [ True]])

In [11]:
a = pred.numpy()
b = y_data.numpy()
pred.numpy().reshape(-1).shape, y_data.numpy().shape

((569,), (569,))

In [12]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(a,b))

              precision    recall  f1-score   support

       False       0.98      0.93      0.95       377
        True       0.87      0.96      0.92       192

    accuracy                           0.94       569
   macro avg       0.93      0.95      0.93       569
weighted avg       0.94      0.94      0.94       569

